In [ ]:
# this script combines our 30.01_quaternary steering, traffic-light stopping, and a velocity pid
# we self-steer with the steering classifier, we stop hard on red with a debounced release, and we track speed via pid
# we keep the hud from our traffic model driving, but cleaner
#
# NEW: Press 'w' to temporarily disable the red-light "hard stop" for a few seconds (configurable).
#
# controls:
#   q/e/r  = left / right / cancel turn signal
#   p      = respawn vehicle
#   t      = force tls => parallel green, perpendicular red (frozen)
#   g      = force tls => parallel red, perpendicular green (frozen)
#   u      = un-freeze all traffic lights (resume cycling)
#   w      = temporary override: disable hard stop for a few seconds
#   esc    = quit
#
# expected models in ../Models/ :
#   30.01_quaternary_model_final.pth
#   traffic_classifier_state.pth  (+ class_mapping.json optional)
#   velocity_regressor_split.pth

import os
import json
import time
import random
from pathlib import Path

import numpy as np
import pygame
import carla

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
from carla import VehicleLightState, TrafficLightState
import cv2

# this part defines our steering classifier. resnet18 backbone, small head, plus a learned turn-signal embedding
CLASS_NAMES_STEER = [
    "Left Hardest", "Left Harder", "Left Hard", "Left Medium", "Left Light", "Left Slight", "Left Minimal",
    "No Turning",
    "Right Minimal", "Right Slight", "Right Light", "Right Medium", "Right Hard", "Right Harder", "Right Hardest"
]

class SteeringClassifier(nn.Module):
    def __init__(self, num_classes=15):
        super().__init__()
        try:
            resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        except Exception:
            resnet = models.resnet18(pretrained=True)
        resnet.fc = nn.Identity()
        self.cnn = resnet
        self.turn_embed = nn.Embedding(3, 16)
        self.fc = nn.Sequential(
            nn.Linear(512 + 16, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, image, turn_signal):
        x_img = self.cnn(image)
        x_signal = self.turn_embed(turn_signal)
        x = torch.cat((x_img, x_signal), dim=1)
        return self.fc(x)

# this model regresses target speed from the camera. simple resnet18 head to one scalar
class VelocityRegression(nn.Module):
    def __init__(self):
        super().__init__()
        try:
            base = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        except Exception:
            base = models.resnet18(pretrained=True)
        base.fc = nn.Linear(base.fc.in_features, 1)
        self.model = base

    def forward(self, image):
        return self.model(image)

# this part sets up our traffic-light classifier. resnet18 with a dropout + linear head
SAVE_DIR       = "../Models"
WEIGHTS_PTH_TL = os.path.join(SAVE_DIR, "traffic_classifier_state.pth")
CLASS_MAP_JSON = os.path.join(SAVE_DIR, "class_mapping.json")

label_to_index = {-1: 0, 0: 1, 1: 2}
index_to_label = {v: k for k, v in label_to_index.items()}
CLASS_NAMES_TL = {-1: "No Light", 0: "Red", 1: "Green"}

if os.path.exists(CLASS_MAP_JSON):
    with open(CLASS_MAP_JSON, "r") as f:
        m = json.load(f)
    if "label_to_index" in m and "index_to_label" in m:
        label_to_index = {int(k): int(v) for k, v in m["label_to_index"].items()}
        index_to_label = {int(v): int(k) for k, v in label_to_index.items()}  # keep inverse consistent

NUM_CLASSES_TL = 3
IN_CHANNELS_TL = 3  # camera is rgb

class TLResNetClassifier(nn.Module):
    def __init__(self, in_channels=3, num_classes=3):
        super().__init__()
        try:
            from torchvision.models import resnet18
            backbone = resnet18(weights=None)
        except Exception:
            from torchvision.models import resnet18
            backbone = resnet18(pretrained=False)

        if in_channels != 3:
            backbone.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        in_feat = backbone.fc.in_features
        backbone.fc = nn.Sequential(nn.Dropout(0.2), nn.Linear(in_feat, num_classes))
        self.net = backbone

    def forward(self, x):
        if x.dim() == 4:
            _, _, H, W = x.shape
            if H < 224 or W < 224:
                x = F.interpolate(x, size=(224, 224), mode="bilinear", align_corners=False)
        return self.net(x)

# we initialize all three models, load weights, and set eval
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

steer_model = SteeringClassifier(num_classes=15).to(device)
steer_model.load_state_dict(torch.load(os.path.join(SAVE_DIR, "30.01_quaternary_model_final.pth"), map_location=device))
steer_model.eval()

velocity_model = VelocityRegression().to(device)
velocity_model.load_state_dict(torch.load(os.path.join(SAVE_DIR, "velocity_regressor_split.pth"), map_location=device))
velocity_model.eval()

tl_model = TLResNetClassifier(IN_CHANNELS_TL, NUM_CLASSES_TL).to(device)
if not os.path.exists(WEIGHTS_PTH_TL):
    print(f"[warn] traffic-light weights not found at {WEIGHTS_PTH_TL}. overlay will show n/a and red-stop disabled.")
    TL_AVAILABLE = False
else:
    state = torch.load(WEIGHTS_PTH_TL, map_location=device)
    tl_state_dict = state["model_state_dict"] if isinstance(state, dict) and "model_state_dict" in state else state
    tl_model.load_state_dict(tl_state_dict, strict=True)
    TL_AVAILABLE = True
tl_model.eval()

# shared transforms for steering and velocity paths
image_transform_steervel = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

# this helper predicts tl class from an rgb frame. returns name and confidence. no emojis
@torch.no_grad()
def predict_traffic_light(np_rgb):
    if not TL_AVAILABLE:
        return "N/A", 0.0

    x = np.asarray(np_rgb)
    if x.ndim == 2:
        x = x[..., None]
    if x.dtype != np.float32:
        x = x.astype(np.float32)
    if x.max() > 1.5:
        x /= 255.0

    x = np.transpose(x, (2, 0, 1))

    if x.shape[0] != IN_CHANNELS_TL:
        if IN_CHANNELS_TL == 3 and x.shape[0] == 1:
            x = np.repeat(x, 3, axis=0)
        elif IN_CHANNELS_TL == 1 and x.shape[0] == 3:
            x = x.mean(axis=0, keepdims=True)
        else:
            raise ValueError(f"channel mismatch for tl model: got {x.shape[0]}, expected {IN_CHANNELS_TL}")

    xt = torch.from_numpy(x).unsqueeze(0).to(device)
    _, _, H, W = xt.shape
    if H != 224 or W != 224:
        xt = F.interpolate(xt, size=(224, 224), mode="bilinear", align_corners=False)

    mean = torch.tensor(IMAGENET_MEAN, device=device).view(1, 3, 1, 1)
    std  = torch.tensor(IMAGENET_STD,  device=device).view(1, 3, 1, 1)
    xt = (xt - mean) / std

    logits = tl_model(xt)
    probs = torch.softmax(logits, dim=1).squeeze(0)
    conf, pred_idx = torch.max(probs, dim=0)

    pred_label = index_to_label.get(int(pred_idx.item()), -1)
    name  = CLASS_NAMES_TL.get(pred_label, str(pred_label))
    return name, float(conf.item())

# we bring up pygame and carla. we spawn the car and two cameras
pygame.init()
width, height = 1280, 800
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("carla: steering + tl + velocity pid (hud)")

font_sm  = pygame.font.SysFont("segoe ui", 22)
font     = pygame.font.SysFont("segoe ui", 28)
font_big = pygame.font.SysFont("segoe ui", 36)

client = carla.Client("localhost", 2000)
client.set_timeout(25.0)
client.load_world("Town05")
world = client.get_world()
bp = world.get_blueprint_library()

vehicle_bp = (bp.filter("vehicle.dodge.charger_2020") or bp.filter("vehicle.*model3*"))[0]
spawn_point = random.choice(world.get_map().get_spawn_points())
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
vehicle.set_autopilot(False)

spectator = world.get_spectator()

# narrow front camera (steer + vel)
cam_bp = bp.find('sensor.camera.rgb')
cam_bp.set_attribute('image_size_x', '640')
cam_bp.set_attribute('image_size_y', '360')
cam_bp.set_attribute('fov', '145')
cam_bp.set_attribute('sensor_tick', '0.1')
cam_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(cam_bp, cam_transform, attach_to=vehicle)

# wide hud / tl camera
cam_bp2 = bp.find('sensor.camera.rgb')
cam_bp2.set_attribute('image_size_x', '1280')
cam_bp2.set_attribute('image_size_y', '800')
cam_bp2.set_attribute('fov', '110')
cam_bp2.set_attribute('sensor_tick', '0.02')
cam_transform2 = carla.Transform(carla.Location(x=1.5, z=2.4))
camera2 = world.spawn_actor(cam_bp2, cam_transform2, attach_to=vehicle)

camera_np = None
camera_np2 = None
camera_np3 = None
pred_label_steer = "Loading..."
pred_label_tl = "N/A"
conf_tl = 0.0

# we crop for tl: center zoom, blackout bottom, soft side blur+desat
def traffic_light_crop(img: np.ndarray) -> np.ndarray:
    if img is None or img.ndim != 3:
        return img

    h, w = img.shape[:2]
    out = img

    z = 1.0 + (70.0 / 100.0)
    new_w = max(1, int(w / z))
    new_h = max(1, int(h / z))
    x1 = max(0, (w - new_w) // 2)
    y1 = max(0, (h - new_h) // 2)
    cropped = out[y1:y1 + new_h, x1:x1 + new_w]
    if cropped.size > 0:
        out = cv2.resize(cropped, (w, h))

    out[h * 2 // 3 :, ...] = 0

    quarter_w = max(1, w // 4)
    def blur_and_desaturate(region: np.ndarray) -> np.ndarray:
        blurred = cv2.GaussianBlur(region, (21, 21), 0)
        hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
        hsv[:, :, 1] = 0
        return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    out[:, :quarter_w, :] = blur_and_desaturate(out[:, :quarter_w, :])
    out[:, -quarter_w:, :] = blur_and_desaturate(out[:, -quarter_w:, :])

    return out

# camera listeners convert raw buffers to numpy. we keep surfaces local in the hud draw
def process_image(image):
    global camera_np
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    array = array[:, :, :3][:, :, ::-1]
    camera_np = array.copy()

def process_image2(image):
    global camera_np2, camera_np3
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    array = array[:, :, :3][:, :, ::-1]
    camera_np2 = traffic_light_crop(array)
    camera_np3 = array.copy()

camera.listen(process_image)
camera2.listen(process_image2)

# steering class to steering value. we shape it later by speed gain
steering_map = {
    0: -0.8,   1: -0.6,   2: -0.4,   3: -0.25,  4: -0.20,  5: -0.10,  6: -0.05,
    7:  0.0,
    8:  0.05,  9:  0.10, 10:  0.15, 11:  0.25, 12:  0.40, 13:  0.60, 14:  0.80
}

# velocity pid gains and clamps
PROPORTIONAL_GAIN = 0.7
INTEGRAL_GAIN     = 0.02
DERIVATIVE_GAIN   = 0.28
INTEGRAL_CLAMP    = 5.0
MAX_ACCEL_THROTTLE = 1.0
MAX_BRAKE = 1.0
BRAKE_GAIN_MULT   = 2.2
MIN_BRAKE_TO_BITE = 0.15
IGNORE_ERROR      = 0.03

integral_err = 0.0
prev_err = 0.0
prev_time = time.time()

# hard stop thresholds for red. we also release on sudden confidence drop
RED_CONF_THRESHOLD     = 0.73
RELEASE_CONF_THRESHOLD = 0.69
CONF_DROP_RELEASE      = 0.05
hard_stop_active = False
prev_conf_tl = None

# NEW: override window triggered by 'w' to disable hard stop
hard_stop_override_until = 0.0         # epoch seconds until which hard stop is ignored
HARD_STOP_OVERRIDE_SEC   = 5.0         # duration of override (seconds)

# internal left-turn tracking. hud label removed
left_signal_on = False
left_turn_tracking_active = False
left_turn_deg = 0.0
left_turn_prev_yaw = None
prev_left_signal_on = False

def _wrap_deg(d: float) -> float:
    return (d + 180.0) % 360.0 - 180.0

def _delta_deg(prev: float, cur: float) -> float:
    return _wrap_deg(cur - prev)

def get_speed(actor: carla.Actor) -> float:
    v = actor.get_velocity()
    return float((v.x**2 + v.y**2 + v.z**2)**0.5)

# we shape steering by speed. more boost at very low speeds, more damping at high speeds
BOOST_END_V         = 5.0
BOOST_GAIN_AT_0V    = 3.0
STEER_GAIN_HIGH_V   = 20.0
STEER_GAIN_HIGH     = 0.35

def _lerp(a: float, b: float, t: float) -> float:
    t = max(0.0, min(1.0, t))
    return a + (b - a) * t

def _speed_gain(v: float) -> float:
    if v <= 0.0:
        return BOOST_GAIN_AT_0V
    if v <= BOOST_END_V:
        t = v / BOOST_END_V
        return _lerp(BOOST_GAIN_AT_0V, 1.0, t)
    if v >= STEER_GAIN_HIGH_V:
        return STEER_GAIN_HIGH
    t = (v - BOOST_END_V) / (STEER_GAIN_HIGH_V - BOOST_END_V)
    return _lerp(1.0, STEER_GAIN_HIGH, t)

# helpers to align world tls relative to our yaw. lets us force states for quick tests
PARALLEL_THRESH_DEG = 30.0
PERP_THRESH_DEG     = 45.0

def _abs_wrap_deg(d: float) -> float:
    return abs(_wrap_deg(d))

def _is_parallel(veh_yaw: float, tl_yaw: float) -> bool:
    d0   = _abs_wrap_deg(tl_yaw - veh_yaw)
    d180 = _abs_wrap_deg((tl_yaw - 180.0) - veh_yaw)
    return (d0 <= PARALLEL_THRESH_DEG) or (d180 <= PARALLEL_THRESH_DEG)

def _is_perpendicular(veh_yaw: float, tl_yaw: float) -> bool:
    d90  = _abs_wrap_deg((tl_yaw - 90.0) - veh_yaw)
    d270 = _abs_wrap_deg((tl_yaw - 270.0) - veh_yaw)
    return (d90 <= PERP_THRESH_DEG) or (d270 <= PERP_THRESH_DEG)

def _set_tl_times(tl, green_t=10.0, red_t=10.0, yellow_t=3.0):
    try:
        tl.set_green_time(float(green_t))
        tl.set_red_time(float(red_t))
        tl.set_yellow_time(float(yellow_t))
    except Exception:
        pass

def _apply_tl_state(tl, state: TrafficLightState, hold=True):
    try:
        tl.set_state(state)
        _set_tl_times(tl)
        tl.freeze(hold)
    except Exception:
        pass

def set_perp_red_parallel_green(world, vehicle, hold=True):
    v_yaw = vehicle.get_transform().rotation.yaw
    tls = world.get_actors().filter('traffic.traffic_light*')
    for tl in tls:
        tl_yaw = tl.get_transform().rotation.yaw
        if _is_parallel(v_yaw, tl_yaw):
            _apply_tl_state(tl, TrafficLightState.Green, hold)
        elif _is_perpendicular(v_yaw, tl_yaw):
            _apply_tl_state(tl, TrafficLightState.Red, hold)

def set_perp_green_parallel_red(world, vehicle, hold=True):
    v_yaw = vehicle.get_transform().rotation.yaw
    tls = world.get_actors().filter('traffic.traffic_light*')
    for tl in tls:
        tl_yaw = tl.get_transform().rotation.yaw
        if _is_parallel(v_yaw, tl_yaw):
            _apply_tl_state(tl, TrafficLightState.Red, hold)
        elif _is_perpendicular(v_yaw, tl_yaw):
            _apply_tl_state(tl, TrafficLightState.Green, hold)

def unfreeze_all_lights(world):
    tls = world.get_actors().filter('traffic.traffic_light*')
    for tl in tls:
        try:
            tl.freeze(False)
        except Exception:
            pass

# small hud helpers for panels, text, and bars
def draw_panel(surface, x, y, w, h, alpha=160):
    panel = pygame.Surface((w, h), pygame.SRCALPHA)
    panel.fill((12, 12, 14, alpha))
    surface.blit(panel, (x, y))

def draw_text(surface, text, x, y, color=(255,255,255), fnt=font, shadow=True):
    if shadow:
        s = fnt.render(text, True, (0,0,0))
        surface.blit(s, (x+1, y+1))
    t = fnt.render(text, True, color)
    surface.blit(t, (x, y))

def draw_badge(surface, text, x, y, padding=6):
    t = font_sm.render(text, True, (255,255,255))
    rect = t.get_rect()
    bg = pygame.Surface((rect.width + padding*2, rect.height + padding*2), pygame.SRCALPHA)
    bg.fill((30, 30, 36, 200))
    surface.blit(bg, (x, y))
    surface.blit(t, (x+padding, y+padding))

def draw_bipolar_bar(surface, x, y, w, h, value, vmin=-1.0, vmax=1.0, label=""):
    pygame.draw.rect(surface, (60, 60, 70), (x, y, w, h), border_radius=6)
    midx = x + w//2
    pygame.draw.rect(surface, (90, 90, 100), (midx-1, y, 2, h), border_radius=1)
    v = max(vmin, min(vmax, value))
    if v >= 0:
        pw = int((v / (vmax if vmax != 0 else 1e-6)) * (w//2))
        pygame.draw.rect(surface, (80, 200, 120), (midx, y, pw, h), border_radius=6)
    else:
        pw = int((abs(v) / (abs(vmin) if vmin != 0 else 1e-6)) * (w//2))
        pygame.draw.rect(surface, (220, 100, 90), (midx - pw, y, pw, h), border_radius=6)
    if label:
        draw_text(surface, f"{label}: {value:+.3f}", x, y - 26, (220,220,220), font_sm)

clock = pygame.time.Clock()
print("model-controlled driving started. q/e/r signals, p respawn, t/g force tls, u unfreeze, w override, esc exit.")

# helper to respawn and reattach sensors
def respawn_vehicle():
    global vehicle, camera, camera2
    try:
        camera.stop(); camera.destroy()
        camera2.stop(); camera2.destroy()
    except Exception:
        pass
    try:
        vehicle.destroy()
    except Exception:
        pass

    sp = random.choice(world.get_map().get_spawn_points())
    vehicle_new = world.spawn_actor(vehicle_bp, sp)
    vehicle_new.set_autopilot(False)

    camera_new = world.spawn_actor(cam_bp, cam_transform, attach_to=vehicle_new)
    camera_new.listen(process_image)

    camera_new2 = world.spawn_actor(cam_bp2, cam_transform2, attach_to=vehicle_new)
    camera_new2.listen(process_image2)

    vehicle = vehicle_new
    camera = camera_new
    camera2 = camera_new2

# main loop. we predict, control, render, and listen for hotkeys
try:
    prev_steering_value = 0.0
    last_steering_time = time.time()
    STEERING_HOLD_S = 0.05

    while True:
        clock.tick(60)
        pygame.event.pump()
        keys = pygame.key.get_pressed()

        # turn signals q / e / r
        if keys[pygame.K_q]:
            vehicle.set_light_state(VehicleLightState.LeftBlinker)
            turn_signal = -1
        elif keys[pygame.K_e]:
            vehicle.set_light_state(VehicleLightState.RightBlinker)
            turn_signal = 1
        elif keys[pygame.K_r]:
            vehicle.set_light_state(VehicleLightState.NONE)
            turn_signal = 0
        else:
            ls = vehicle.get_light_state()
            if ls & VehicleLightState.LeftBlinker:
                turn_signal = -1
            elif ls & VehicleLightState.RightBlinker:
                turn_signal = 1
            else:
                turn_signal = 0

        if keys[pygame.K_p]:
            print("respawning vehicle...")
            respawn_vehicle()
            time.sleep(0.5)
            continue

        # track signal edges for internal left-turn yaw accumulation
        ls_now = vehicle.get_light_state()
        left_signal_on  = bool(ls_now & VehicleLightState.LeftBlinker) or (turn_signal == -1)
        right_signal_on = bool(ls_now & VehicleLightState.RightBlinker) or (turn_signal == 1)

        if not left_signal_on and not right_signal_on:
            left_turn_deg = 0.0
            left_turn_tracking_active = False
            left_turn_prev_yaw = None

        if right_signal_on:
            left_turn_deg = 0.0

        if left_signal_on and not prev_left_signal_on:
            left_turn_tracking_active = True
            left_turn_deg = 0.0
            left_turn_prev_yaw = vehicle.get_transform().rotation.yaw
        elif (not left_signal_on) and prev_left_signal_on:
            left_turn_tracking_active = False
            left_turn_prev_yaw = None
        prev_left_signal_on = left_signal_on

        control = carla.VehicleControl()

        # predict steering + velocity
        target_speed = None
        predicted_speed = None
        if camera_np is not None:
            with torch.no_grad():
                img_t = image_transform_steervel(camera_np).unsqueeze(0).to(device)

                signal_t = torch.tensor([[turn_signal + 1]], dtype=torch.long).to(device)
                out = steer_model(img_t, signal_t.squeeze(1))
                pred_class = out.argmax(dim=1).item()
                pred_label_steer = CLASS_NAMES_STEER[pred_class]
                steering_value = steering_map[pred_class]

                # speed-based shaping
                current_speed_for_gain = get_speed(vehicle)
                gain = _speed_gain(current_speed_for_gain)
                steering_value *= gain
                steering_value = max(-1.0, min(1.0, steering_value))

                # small bias to honor blinkers
                if turn_signal == -1 and steering_value > -0.03:
                    steering_value = -0.03
                elif turn_signal == 1 and steering_value < 0.07:
                    steering_value = 0.07

                # hold to reduce chatter
                now = time.time()
                if steering_value == 0.0 or prev_steering_value == 0.0:
                    control.steer = steering_value
                    prev_steering_value = steering_value
                    last_steering_time = now
                elif steering_value != prev_steering_value:
                    control.steer = (steering_value + prev_steering_value) / 2
                    prev_steering_value = steering_value
                    last_steering_time = now
                else:
                    if now - last_steering_time >= STEERING_HOLD_S:
                        control.steer = steering_value

                # velocity regression (m/s), clamp to a safe band
                v_out = velocity_model(img_t)
                raw_speed = float(v_out.squeeze().item())
                predicted_speed = max(2.5, min(8.5, raw_speed))
                target_speed = predicted_speed

        # predict tl and manage hard stop (with W override)
        if camera_np2 is not None:
            with torch.no_grad():
                pred_label_tl, conf_tl = predict_traffic_light(camera_np2)

            name_is_red   = (pred_label_tl == "Red")
            name_is_green = (pred_label_tl == "Green")
            name_is_none  = (pred_label_tl == "No Light")

            if TL_AVAILABLE:
                sudden_drop = (prev_conf_tl is not None) and ((prev_conf_tl - conf_tl) >= CONF_DROP_RELEASE)
                now = time.time()
                if now < hard_stop_override_until:
                    # override active: ignore hard stop, but still track prev_conf
                    hard_stop_active = False
                else:
                    if name_is_red and conf_tl >= RED_CONF_THRESHOLD:
                        hard_stop_active = True
                    elif hard_stop_active and (sudden_drop or (not name_is_red) or conf_tl < RELEASE_CONF_THRESHOLD):
                        hard_stop_active = False
            prev_conf_tl = conf_tl

        # pid throttle/brake or hard stop
        throttle_cmd = 0.0
        brake_cmd = 0.0

        if hard_stop_active:
            throttle_cmd = 0.0
            brake_cmd = 1.0
            try:
                ls = vehicle.get_light_state()
                vehicle.set_light_state(ls | VehicleLightState.Brake)
            except Exception:
                pass
        else:
            if target_speed is not None:
                current_speed = get_speed(vehicle)
                speed_error = target_speed - current_speed

                now_t = time.time()
                dt = max(1e-3, now_t - prev_time)

                if abs(speed_error) < IGNORE_ERROR:
                    throttle_cmd = 0.0
                    brake_cmd = 0.0
                else:
                    integral_err = max(-INTEGRAL_CLAMP, min(INTEGRAL_CLAMP, integral_err + speed_error * dt))
                    rate_of_change = (speed_error - prev_err) / dt

                    pid_output = (
                        PROPORTIONAL_GAIN * speed_error
                        + INTEGRAL_GAIN * integral_err
                        + DERIVATIVE_GAIN * rate_of_change
                    )

                    if pid_output >= 0:
                        throttle_cmd = min(MAX_ACCEL_THROTTLE, max(0.0, pid_output))
                        brake_cmd = 0.0
                    else:
                        throttle_cmd = 0.0
                        brake_cmd = min(MAX_BRAKE, max(0.0, BRAKE_GAIN_MULT * (-pid_output)))
                        if 0.0 < brake_cmd < MIN_BRAKE_TO_BITE:
                            brake_cmd = MIN_BRAKE_TO_BITE
                        integral_err = 0.0

                prev_err = speed_error
                prev_time = now_t
            else:
                throttle_cmd = 0.0
                brake_cmd = 0.0

        control.throttle = throttle_cmd
        control.brake = brake_cmd
        control.hand_brake = False
        vehicle.apply_control(control)

        # spectator follow
        tr = vehicle.get_transform()
        fwd = tr.get_forward_vector()
        cam_location = tr.location - fwd * 8 + carla.Location(z=3)
        cam_rotation = carla.Rotation(pitch=-10, yaw=tr.rotation.yaw)
        spectator.set_transform(carla.Transform(cam_location, cam_rotation))

        # accumulate yaw while left signal is on (internal only)
        if left_turn_tracking_active:
            cur_yaw = tr.rotation.yaw
            if left_turn_prev_yaw is not None:
                d = _delta_deg(left_turn_prev_yaw, cur_yaw)
                left_turn_deg += d
            left_turn_prev_yaw = cur_yaw

        # hud draw. background is the narrow front feed for clarity
        if camera_np3 is not None:
            frame_surface = pygame.surfarray.make_surface(camera_np3.swapaxes(0, 1))
            frame_surface = pygame.transform.scale(frame_surface, (width, height))
            screen.blit(frame_surface, (0, 0))

        # steering panel
        panel_w = 480
        panel_h = 150
        draw_panel(screen, 16, 16, panel_w, panel_h, alpha=170)
        draw_badge(screen, "steering", 24, 20)
        draw_text(screen, f"class: {pred_label_steer}", 30, 60, (240,240,120), font)
        draw_bipolar_bar(screen, 30, 126, panel_w-60, 18, value=float(prev_steering_value), label="steer")

        # speed panel
        vpanel_w = 420
        vpanel_h = 120
        draw_panel(screen, 16, height - vpanel_h - 16, vpanel_w, vpanel_h, alpha=170)
        draw_badge(screen, "speed", 24, height - vpanel_h - 12)
        cur_speed = get_speed(vehicle)
        draw_text(screen, f"current:  {cur_speed:.2f} m/s", 30, height - vpanel_h + 32, (180,220,255), font)
        if 'predicted_speed' in locals() and predicted_speed is not None:
            draw_text(screen, f"predicted: {predicted_speed:.2f} m/s", 30, height - vpanel_h + 70, (180,255,200), font)
        else:
            draw_text(screen, "predicted: -- m/s", 30, height - vpanel_h + 70, (200,200,200), font)

        # traffic light panel
        tpanel_w = 520
        tpanel_h = 120
        draw_panel(screen, width - tpanel_w - 16, height - tpanel_h - 16, tpanel_w, tpanel_h, alpha=170)
        draw_badge(screen, "traffic light", width - tpanel_w - 8 + 16, height - tpanel_h - 12)
        tl_text = f"prediction: {pred_label_tl}"
        conf_text = f"confidence: {conf_tl*100:.1f}%"
        draw_text(screen, tl_text, width - tpanel_w + 20, height - tpanel_h + 32, (255,255,255), font)
        draw_text(screen, conf_text, width - tpanel_w + 20, height - tpanel_h + 70,
                  (255,180,120) if TL_AVAILABLE else (180,180,180), font)
        # if hard_stop_active:
            # draw_badge(screen, "hard stop", width - 150, height - tpanel_h - 8)

        # show override badge/countdown if active
        remaining_override = max(0.0, hard_stop_override_until - time.time())
        if remaining_override > 0.0:
            draw_badge(screen, f"override {remaining_override:.1f}s", width - 200, height - tpanel_h - 44)

        pygame.display.flip()

        # events and hotkeys
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                raise KeyboardInterrupt
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    raise KeyboardInterrupt
                if event.key == pygame.K_t:
                    set_perp_red_parallel_green(world, vehicle, hold=True)
                    print("[tl] set parallel green, perpendicular red (frozen)")
                if event.key == pygame.K_g:
                    set_perp_green_parallel_red(world, vehicle, hold=True)
                    print("[tl] set parallel red, perpendicular green (frozen)")
                if event.key == pygame.K_u:
                    unfreeze_all_lights(world)
                    print("[tl] unfroze all traffic lights")
                if event.key == pygame.K_w:
                    hard_stop_override_until = time.time() + HARD_STOP_OVERRIDE_SEC
                    print(f"[override] hard stop disabled for {HARD_STOP_OVERRIDE_SEC} seconds")

except KeyboardInterrupt:
    print("exiting and cleaning up...")
finally:
    try:
        camera.stop(); camera2.stop()
    except Exception:
        pass
    try:
        vehicle.destroy()
    except Exception:
        pass
    try:
        camera.destroy(); camera2.destroy()
    except Exception:
        pass
    pygame.quit()


model-controlled driving started. q/e/r signals, p respawn, t/g force tls, u unfreeze, w override, esc exit.
[override] hard stop disabled for 5.0 seconds
[override] hard stop disabled for 5.0 seconds
[tl] set parallel red, perpendicular green (frozen)
respawning vehicle...
[tl] set parallel red, perpendicular green (frozen)
[override] hard stop disabled for 5.0 seconds
[override] hard stop disabled for 5.0 seconds
[override] hard stop disabled for 5.0 seconds
exiting and cleaning up...
